In [1]:
### Import libraries
import openai
import os
import time
from datetime import datetime
import pandas as pd
import numpy as np
from numpy.linalg import norm
import psycopg2
import clingo
from clingo.control import Control
import json
from yaml import load, Loader
from helper import *
from stories import *

In [2]:
def asp_try_3(asp_file,instance):    
    errors = []
    messages = []
    models = []  
    symbols = []
    
    def custom_logger(code, message):
        errors.append(code)
        messages.append(message)

    def on_model(model):
        #print("Model:", model)
        models.append(model)
        with open('output.txt','w') as file:
            print(model, file=file)
        with open('output.txt','r') as file:
            lines = file.readlines()
            symbols_list = lines[0].split()
            for s in symbols_list:
                symbols.append(s)
        os.remove("output.txt")
        
    error_codes = ["AtomUndefined","FileIncluded","GlobalVariable","OperationUndefined","Other","RuntimeError","VariableUnbounded"]
    sub_codes = ["unsafe"]
    asp_program = []  
    error_dict = {}
    message_dict = {}
    for error_code in error_codes:
        error_dict[error_code] = 0
    for sub_code in sub_codes:
        message_dict[sub_code] = 0
    
    control = clingo.Control(logger=custom_logger,message_limit=100)
    input_files = [asp_file, instance]   
    for file_name in input_files:
        with open(file_name, "r") as file:
            asp_program.extend(file.readlines())
    try:
        control.add("base", [], "".join(asp_program))
    except Exception as err:         
        print(f"Unexpected {err=}, {type(err)=}.")
        for e in errors:
            for ec in error_codes:
                if str(e).split(".")[1] == ec:
                    error_dict[ec] += 1
        for mss in messages:
            for sc in sub_codes:
                if sc in mss:
                    message_dict[sc] += 1
        return models, error_dict, message_dict, errors, messages, symbols
    try:
        control.ground([("base", [])])
    except Exception as err:         
        print(f"Unexpected {err=}, {type(err)=}.")
        for e in errors:
            for ec in error_codes:
                if str(e).split(".")[1] == ec:
                    error_dict[ec] += 1
        for mss in messages:
            for sc in sub_codes:
                if sc in mss:
                    message_dict[sc] += 1
        return models, error_dict, message_dict, errors, messages, symbols
        
    #control.configuration.solve.models = 0  # Limit the number of models to 1
    try:
        control.solve(on_model=on_model)
    except Exception as err:         
        print(f"Unexpected {err=}, {type(err)=}.")
        for e in errors:
            for ec in error_codes:
                if str(e).split(".")[1] == ec:
                    error_dict[ec] += 1
        for mss in messages:
            for sc in sub_codes:
                if sc in mss:
                    message_dict[sc] += 1
        return models, error_dict, message_dict, errors, messages, symbols
    
    return models, error_dict, message_dict, errors, messages, symbols

In [3]:
canonical_dict = {"sudoku":os.path.join("projects","sudoku","canonical_sudoku.lp"),
                 "seeknumbers":os.path.join("projects","seeknumbers","canonical_seeknumbers2.lp"),
                 "minotaur":os.path.join("projects","minotaur","canonical_minotaur1.lp"),
                 "creek":os.path.join("projects","creek","canonical_creek.lp"),                 
                 "yosenabe":os.path.join("projects","yosenabe","canonical_yosenabe.lp"),
                 "hop":os.path.join("projects","hop","canonical_hop.lp"),
                 "lights":os.path.join("projects","lights","canonical_lights.lp")}

In [4]:
instances_dict ={"sudoku":os.path.join("projects","sudoku","instances","ex01.lp"),
                 "seeknumbers":os.path.join("projects","seeknumbers","instances","ex01.lp"),
                 "minotaur":os.path.join("projects","minotaur","instances","level01.lp"),
                 "creek":os.path.join("projects","creek","instances","ex01.lp"),
                 "yosenabe":os.path.join("projects","yosenabe","instances","instance01.lp"),
                 "hop":os.path.join("projects","hop","instances","level1.lp"),
                 "lights":os.path.join("projects","lights","instances","test01.lp")}

In [5]:
solution_dict ={"sudoku":os.path.join("projects","sudoku","solutions","ex01.json"),
                 "seeknumbers":os.path.join("projects","seeknumbers","solutions","ex01.json"),
                 "minotaur":os.path.join("projects","minotaur","solutions","level01.json"),
                 "creek":os.path.join("projects","creek","solutions","ex01.json"),
                 "yosenabe":os.path.join("projects","yosenabe","solutions","solution01.txt"),
                 "hop":os.path.join("projects","hop","solutions","solution01.txt"),
                 "lights":os.path.join("projects","lights","solutions","solution01.txt")}

In [6]:
def get_symbols_from_solution(solution_path):        
    with open(solution_path,'r') as file:
        if ".json" in solution_path:
            data = json.load(file)
            results = data["Call"][0]["Witnesses"] #[0]["Value"]
            #print(result)
        else:
            lines = file.readlines()
            splitted = lines[0].split()
            results = [{"Value":splitted}]
    return results

In [7]:
def match_counter(symbols, solution):
    matches = 0
    for symbol in symbols:
        if symbol in solution:
            matches += 1
    return matches

In [12]:
def evaluate(file, instance, solution_path, canonical_file):
    models, error_dict, message_dict, errors, messages, symbols = asp_try_3(file,instance)
    n_models = len(models)
    canonical_lines = lines_counter(canonical_file)
    encoding_lines = lines_counter(file)
    #print("Length of Canonical:" + str(canonical_lines) + ". Length Encoding: "+ str(encoding_lines))
    errors = 0
    for element in error_dict:
        errors += error_dict[element]
    diff_lines = abs(canonical_lines - encoding_lines)
    solutions = get_symbols_from_solution(solution_path)
    #print("Symbols: " + str(symbols))
    #print("Solution: " + str(solution))
    matchs = 0
    match_ratio = 0
    for solution in solutions:
        matchs_temp = match_counter(symbols,solution["Value"])
        if matchs_temp > matchs:
            matchs = matchs_temp
            match_ratio = round(matchs/len(solution["Value"]),2)
    kpi = 50 + n_models*25 + match_ratio*25 - diff_lines - errors * 5
    print("KPI = {}. Diff_Lines: {}. Errors: {}. N_models: {}. Symbols: {}. Matchs: {}. Match_Ratio: {}".format(kpi,diff_lines,errors,n_models,len(symbols),matchs, match_ratio))
    return kpi, n_models, errors, diff_lines, models, symbols, matchs, match_ratio

In [13]:
conn_string = "dbname=thesis user=postgres password=postgres"
conn = psycopg2.connect(conn_string)
print("Connection established")

Connection established


In [ ]:
def save_in_database(conn, file, kpi, diff_lines, errors, n_models, symbols, matchs, match_ratio):
    cursor = conn.cursor()
    command = "INSERT INTO results (file, kpi, diff_lines, errors, n_models, symbols, matchs, match_ratio) VALUES ('%(file)s', '%(kpi)s', '%(diff_lines)s', '%(errors)s', '%(n_models)s', '%(symbols)s', '%(matchs)s', '%(match_ratio)s');" % {"file":file, "kpi": kpi, "diff_lines": diff_lines, "errors": errors, "n_models": n_models, "symbols": symbols, "matchs":matchs, "match_ratio":match_ratio}
    try:
        cursor.execute(command)
        conn.commit()
        cursor.close()
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=} Result not inserted.")
        conn.commit()
        cursor.close()
        pass

## Evaluate Canonicals

In [15]:
projects = [sudoku,seeknumbers,minotaur,creek,yosenabe,hop,lights]
for project in projects:
    story = project["story"]
    print("======= Evaluation Canonical of " + story + " ===============")
    canonical_file = canonical_dict[story]
    instances_file = instances_dict[story]
    solution_file = solution_dict[story]
    kpi, n_models, errors, diff_lines, models, symbols, matchs, match_ratio = evaluate(canonical_file,instances_file,solution_file,canonical_file)

======= Evaluation Canonical of sudoku ===============
KPI = 100.0. Diff_Lines: 0. Errors: 0. N_models: 1. Symbols: 81. Matchs: 81. Match_Ratio: 1.0
======= Evaluation Canonical of seeknumbers ===============
KPI = 78.0. Diff_Lines: 0. Errors: 0. N_models: 1. Symbols: 1. Matchs: 1. Match_Ratio: 0.12
======= Evaluation Canonical of minotaur ===============
KPI = 100.0. Diff_Lines: 0. Errors: 0. N_models: 1. Symbols: 5. Matchs: 5. Match_Ratio: 1.0
======= Evaluation Canonical of creek ===============
KPI = 100.0. Diff_Lines: 0. Errors: 0. N_models: 1. Symbols: 6. Matchs: 6. Match_Ratio: 1.0
======= Evaluation Canonical of yosenabe ===============
KPI = 100.0. Diff_Lines: 0. Errors: 0. N_models: 1. Symbols: 5. Matchs: 5. Match_Ratio: 1.0
======= Evaluation Canonical of hop ===============
KPI = 100.0. Diff_Lines: 0. Errors: 0. N_models: 1. Symbols: 3. Matchs: 3. Match_Ratio: 1.0
======= Evaluation Canonical of lights ===============
KPI = 100.0. Diff_Lines: 0. Errors: 0. N_models: 1. Symb

## Evaluation of a Version

In [25]:
version = 9

In [26]:
files = os.listdir("generated_solutions")
selected_files = [x for x in files if "v"+str(version) in x]
print(selected_files)

['creek_from_seeknumbers_and_lights_v9_20231108.lp', 'hop_from_seeknumbers_and_lights_v9_20231108.lp', 'lights_from_seeknumbers_and_hop_v9_20231108.lp', 'minotaur_from_seeknumbers_and_hop_v9_20231108.lp', 'seeknumbers_from_creek_and_hop_v9_20231108.lp', 'sudoku_from_yosenabe_and_lights_v9_20231108.lp', 'yosenabe_from_seeknumbers_and_sudoku_v9_20231108.lp']


In [27]:
list_of_models = []
for file in selected_files:
    story = file.split("_")[0]
    canonical_file = canonical_dict[story]
    print("======= Evaluation of " + story + " ===============")
    print(file)
    path_of_file = os.path.join("generated_solutions",file)
    kpi, success, errors, diff_lines, models, symbols, matchs, match_ratio = evaluate(path_of_file,instances_dict[story],solution_dict[story],canonical_file)
    list_of_models.append(models)

======= Evaluation of creek ===============
creek_from_seeknumbers_and_lights_v9_20231108.lp
Unexpected err=RuntimeError('parsing failed'), type(err)=<class 'RuntimeError'>.
KPI = 23.0. Diff_Lines: 22. Errors: 1. N_models: 0. Symbols: 0. Matchs: 0. Match_Ratio: 0.0
======= Evaluation of hop ===============
hop_from_seeknumbers_and_lights_v9_20231108.lp
KPI = 67.75. Diff_Lines: 24. Errors: 0. N_models: 1. Symbols: 33. Matchs: 2. Match_Ratio: 0.67
======= Evaluation of lights ===============
lights_from_seeknumbers_and_hop_v9_20231108.lp
Unexpected err=RuntimeError('grounding stopped because of errors'), type(err)=<class 'RuntimeError'>.
KPI = 26.0. Diff_Lines: 9. Errors: 3. N_models: 0. Symbols: 0. Matchs: 0. Match_Ratio: 0.0
======= Evaluation of minotaur ===============
minotaur_from_seeknumbers_and_hop_v9_20231108.lp
Unexpected err=RuntimeError('grounding stopped because of errors'), type(err)=<class 'RuntimeError'>.
KPI = -3.0. Diff_Lines: 38. Errors: 3. N_models: 0. Symbols: 0. Mat

## Evaluation of a all versions

In [17]:
n_versions = 9

In [ ]:
files = os.listdir("generated_solutions")
data = []
for version in range(1, n_versions+1):
    selected_files = [x for x in files if "v"+str(version) in x in x]
    for file in selected_files:
        story = file.split("_")[0]
        canonical_file = canonical_dict[story]
        print("======= Evaluation of " + story + " ===============")
        #print(file)
        path_of_file = os.path.join("generated_solutions",file)
        kpi, success, errors, diff_lines, models, symbols, matchs, match_ratio = evaluate(path_of_file,instances_dict[story],solution_file,canonical_file)
        save_in_database(conn, file,kpi,diff_lines,errors,n_models,symbols,matchs, match_ratio)
        dict_element = {'story':story, 'file':file, 'kpi':kpi, 'success':success, 'errors':errors, 'diff_lines':diff_lines, 'version':version, 'symbols':symbols,'matchs':matchs}
        data.append(dict_element)

======= Evaluation of creek ===============
KPI = 36. Diff_Lines: 14. Errors: 0. N_models: 0. Symbols: 0. Matchs: 0. Match_Ratio: 0
INSERT INTO evaluations (file, kpi, diff_lines, errors, n_models, symbols, matchs, match_ratio) VALUES ('creek_from_minotaur_and_seeknumbers_v1.lp', '36', '14', '0', '1', '[]', '0', '0');
Unexpected err=UndefinedTable('FEHLER:  Relation »evaluations« existiert nicht\nLINE 1: INSERT INTO evaluations (file, kpi, diff_lines, errors, n_mo...\n                    ^\n'), type(err)=<class 'psycopg2.errors.UndefinedTable'> Result not inserted.
======= Evaluation of creek ===============
KPI = 49. Diff_Lines: 26. Errors: 0. N_models: 1. Symbols: 0. Matchs: 0. Match_Ratio: 0
INSERT INTO evaluations (file, kpi, diff_lines, errors, n_models, symbols, matchs, match_ratio) VALUES ('creek_from_seeknumbers_and_lights_v1_20231111.lp', '49', '26', '0', '1', '[]', '0', '0');
Unexpected err=UndefinedTable('FEHLER:  Relation »evaluations« existiert nicht\nLINE 1: INSERT INTO e

In [14]:
df = pd.DataFrame.from_records(data)
df.sort_values(by='kpi',ascending=False)

,story,file,kpi,success,errors,diff_lines,version,story_type,symbols
3,minotaur,minotaur_from_sudoku_and_creek_v1_s1.lp,92,1,0,8,1,1,[]
25,seeknumbers,seeknumbers_from_sudoku_and_creek_v5_s1.lp,91,1,0,9,5,1,"[cell(1,1), cell(1,2), cell(1,3), cell(2,1), c..."
6,sudoku,sudoku_from_creek_and_yosenabe_v1_s1.lp,91,1,0,9,1,1,[]
8,yosenabe,yosenabe_from_sudoku_and_creek_v1_s1.lp,91,1,0,9,1,1,[]
23,minotaur,minotaur_from_sudoku_and_creek_v5_s1.lp,86,1,0,14,5,1,"[goal(3,2), field(1,1), field(1,2), field(1,3)..."
18,minotaur,minotaur_from_sudoku_and_creek_v4_s1.lp,67,1,0,33,4,1,"[at(2,1,0), at(2,3,0)]"
17,creek,creek_from_sudoku_and_seeknumbers_v4_s1.lp,48,0,0,2,4,1,[]
12,sudoku,sudoku_from_seeknumbers_and_creek_v2_s1.lp,42,0,1,3,2,1,[]
27,sudoku,sudoku_from_seeknumbers_and_creek_v5_s1.lp,34,0,2,6,5,1,[]
16,sudoku,sudoku_from_seeknumbers_and_creek_v3_s1.lp,31,0,1,14,3,1,[]


In [12]:
df.groupby(['version'])['kpi'].mean()

version
1    26.25
2    20.75
3     2.25
4    27.50
5     8.75
Name: kpi, dtype: float64

In [14]:
df.groupby(['story'])['kpi'].mean()

story
creek          13.833333
minotaur       13.833333
seeknumbers    12.333333
sudoku         22.833333
Name: kpi, dtype: float64

In [26]:
file = "minotaur_from_sudoku_and_creek_v5_s1.lp"
with open(os.path.join('generated_solutions',file),'r') as toread:
    print(toread.read()) 

% Define the possible directions to move
direction(up).
direction(down).
direction(left).
direction(right).

% Define the possible movements of the Minotaur based on the current position
minotaur_move(X,Y,X+2,Y) :- minotaur(X,Y), not wall(X+1,Y), not wall(X+2,Y).
minotaur_move(X,Y,X-2,Y) :- minotaur(X,Y), not wall(X-1,Y), not wall(X-2,Y).
minotaur_move(X,Y,X,Y+2) :- minotaur(X,Y), not wall(X,Y+1), not wall(X,Y+2).
minotaur_move(X,Y,X,Y-2) :- minotaur(X,Y), not wall(X,Y-1), not wall(X,Y-2).

% Define the possible movements of the player based on the current position
player_move(X,Y,X+1,Y) :- player(X,Y), not wall(X+1,Y).
player_move(X,Y,X-1,Y) :- player(X,Y), not wall(X-1,Y).
player_move(X,Y,X,Y+1) :- player(X,Y), not wall(X,Y+1).
player_move(X,Y,X,Y-1) :- player(X,Y), not wall(X,Y-1).

% Define the goal state where the player is at the goal position
goal :- player(X,Y), goal(X,Y).

% Define the possible moves of the player and Minotaur
move :- player_move(X,Y,X1,Y1), minotaur_move(X2,Y

## Comparison of lines

In [4]:
projects = [sudoku,seeknumbers,minotaur,creek,yosenabe,hop,lights]

In [5]:
files = os.listdir("generated_solutions")

In [8]:
def lines_counter(file):
    with open(file,'r') as readfile:
        lines = readfile.readlines()
    return len(lines)

In [21]:
for project in projects:
    for version in range(6,9):
        selected_files = [x for x in files if "v"+str(version) in x and (story + "_from") in x]
        #print(selected_files)
        for file in selected_files:
            path_of_file = os.path.join("generated_solutions",file)
            story = project['story']
            print("Version: " + str(version) + ". Story: " + story + ". Lines: " + str(lines_counter(path_of_file)))

Version: 6. Story: sudoku. Lines: 24
Version: 6. Story: sudoku. Lines: 23
Version: 7. Story: sudoku. Lines: 11
Version: 8. Story: sudoku. Lines: 2
Version: 6. Story: seeknumbers. Lines: 19
Version: 6. Story: seeknumbers. Lines: 26
Version: 6. Story: seeknumbers. Lines: 14
Version: 7. Story: seeknumbers. Lines: 57
Version: 8. Story: seeknumbers. Lines: 14
Version: 6. Story: minotaur. Lines: 21
Version: 6. Story: minotaur. Lines: 32
Version: 6. Story: minotaur. Lines: 20
Version: 7. Story: minotaur. Lines: 13
Version: 8. Story: minotaur. Lines: 7
Version: 6. Story: creek. Lines: 31
Version: 6. Story: creek. Lines: 13
Version: 6. Story: creek. Lines: 16
Version: 7. Story: creek. Lines: 26
Version: 8. Story: creek. Lines: 23
Version: 6. Story: yosenabe. Lines: 23
Version: 6. Story: yosenabe. Lines: 42
Version: 6. Story: yosenabe. Lines: 23
Version: 7. Story: yosenabe. Lines: 17
Version: 8. Story: yosenabe. Lines: 8
Version: 6. Story: hop. Lines: 17
Version: 6. Story: hop. Lines: 17
Version

In [20]:
file = "yosenabe_from_creek_and_sudoku_v8_20231102.lp"
with open(os.path.join('generated_solutions',file),'r') as toread:
    print(toread.read()) 

% A number can be moved from one cell to another
possible_target(X,Y,P,Q) :- cell(X,Y), cell(P,Q), number(X,Y,_), not number(P,Q,_).

% A number can be moved into an area if it belongs to that area
possible_target(X,Y,P,Q) :- cell(X,Y), cell(P,Q), number(X,Y,_), area(P,Q,_).

% The sum of the numbers in an area must be equal to the goal
possible_target(X,Y,P,Q) :- cell(X,Y), cell(P,Q), number(X,Y,_), area(P,Q,A), goal(A,G), sum_numbers(A,G).

% The ways of any two moved numbers must not cross or meet at any grid cell
possible_target(X,Y,P,Q) :- cell(X,Y), cell(P,Q), number(X,Y,_), number(P,Q,_), not cross_or_meet(X,Y,P,Q).

% Each gray area must be populated with at least one moved number
possible_target(X,Y,P,Q) :- cell(X,Y), cell(P,Q), number(X,Y,_), area(P,Q,A), gray_area(A), at_least_one_number(A).

% A move stops at the first cell w.r.t. its direction of the area into which it leads
possible_target(X,Y,P,Q) :- cell(X,Y), cell(P,Q), number(X,Y,_), area(P,Q,A), direction(A,D), stop_